In [32]:
""" PLEASE DO NOT DIRECTLY EDIT THIS FILE.

    This version of the U-Net is meant to replicate the sea ice
    segmentation framework as found in GitHub written in Tensorflow
    using PyTorch and the orginal U-Net framework.
    
    Original sea ice repo: https://github.com/asylve/Sea-Ice/blob/main/Masking-Modelling.ipynb
        
    Original U-Net described here: https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/
    Many of the functions have been copied and slightly modified with comments from https://github.com/milesial/Pytorch-UNet
    The new functions are for the Dataset class and for the create_npy_list function. 
    
    The original U-Net file structure was split into multiple module files. For this working draft, 
    all functions have been incorporated into this single file in seperate cells along with their package dependancies
    so that splitting into seperate modules in the future, if desired, will be simple.
    
    Additionally, this version has been slightly adapted for the single band SAR tile input compared to
    the original 3 band optical image (RGB) input of U-NET. 
    
    Compared to the U-Net, the sea ice segmentation framework: 
        - Does not use BatchNorm 
        - Uses Dropout functions throughout
        - Concatantes the previous layer and current convoluted layer without bespoke padding
          and in slightly different order.
        - Softmax transformation in out convolutional layer.
        - Begins with 32 out channels rather than 64 and scales to 512 rather than >1000
    
    To adjust, must change: dir_image, dir_checkpoint, n_channels, n_classes and bilinear variables.
"""


' This version of the U-Net is meant to replicate the sea ice\n    segmentation framework as found in GitHub written in Tensorflow\n    using PyTorch and the orginal U-Net framework.\n    \n    Original sea ice repo: https://github.com/asylve/Sea-Ice/blob/main/Masking-Modelling.ipynb\n        \n    Original U-Net described here: https://lmb.informatik.uni-freiburg.de/people/ronneber/u-net/\n    Many of the functions have been copied and slightly modified with comments from https://github.com/milesial/Pytorch-UNet\n    The new functions are for the Dataset class and for the create_npy_list function. \n    \n    The original U-Net file structure was split into multiple module files. For this working draft, \n    all functions have been incorporated into this single file in seperate cells along with their package dependancies\n    so that splitting into seperate modules in the future, if desired, will be simple.\n    \n    Additionally, this version has been slightly adapted for the singl

In [9]:
""" Parts of the U-Net model """

import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool, dropout then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            nn.Dropout(p=0.5, inplace=True),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling, single conv, concatination, dropout, then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False),
            #nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True)
        )
        self.dconv = DoubleConv(in_channels, out_channels, in_channels // 2)
        self.dropout = nn.Dropout(p=0.5, inplace=True)
            
    def forward(self, x1, x2):
        x1 = self.up(x1)
        x1 = self.conv(x1)
        
        x = torch.cat([x1, x2], dim=1)
        #print(x1.size(), x2.size(), x.size())
        x = self.dropout(x)
        return self.dconv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.outconv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1),
            nn.Softmax() #softmax converts the output to a list of probabilities that must sum to 1
        )

    def forward(self, x):
        return self.outconv(x)

In [2]:
""" Full assembly of the parts to form the complete network """

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = DoubleConv(n_channels, 32)
        self.down1 = Down(32, 64)
        self.down2 = Down(64, 128)
        self.down3 = Down(128, 256)
        self.down4 = Down(256, 512)
        self.up1 = Up(512, 256, bilinear)
        self.up2 = Up(256, 128, bilinear)
        self.up3 = Up(128, 64, bilinear)
        self.up4 = Up(64, 32, bilinear)
        self.outc = OutConv(32, n_classes)

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [10]:
""" CNN Dataset preparation functions """

import torch
from torch.utils.data.dataset import Dataset  # For custom data-sets
from torchvision import transforms
import glob
import numpy as np
torch.manual_seed(2022)  # Setting random seed so that augmentations can be reproduced.
# From https://discuss.pytorch.org/t/beginner-how-do-i-write-a-custom-dataset-that-allows-me-to-return-image-and-its-target-image-not-label-as-a-pair/13988/4
# And https://discuss.pytorch.org/t/how-make-customised-dataset-for-semantic-segmentation/30881


def create_npy_list(image_directory, img_string="sar"):
    """A function that returns a list of the names of the SAR/MODIS and labelled .npy files in a directory. These lists can
    then be used as an argument for the Dataset class instantiation. The function also checks that the specified directory 
    contains matching sar or MODIS/labelled pairs -- specifically, a label.npy file for each image file."""

    img_names = sorted(glob.glob(str(image_directory) + '/*_' + img_string + '.npy'))
    label_names = sorted(glob.glob(str(image_directory) + '/*_labels.npy'))

    # In-depth file-by-file check for matching sar-label pairs in the directory -- assuming  each sar image has a corresponding
    # labeled image.
    img_label_pairs = []
    for image in img_names:
        expected_label_name = image.replace(img_string, "labels")
        if expected_label_name in label_names:
            img_label_pairs.append((image, expected_label_name))
        else:
            raise Exception(f'{img_string} tile name {image} does not have a matching labeled tile.')
            
    return img_label_pairs


class CustomImageDataset(Dataset):
    """GTC Code for a dataset class. The class is instantiated with list of filenames within a directory (created using
    the list_npy_filenames function). The __getitem__ method pairs up corresponding image-label .npy file pairs. This
    dataset can then be input to a dataloader."""
    
    def __init__(self, paths, isSingleBand = True):
        self.paths = paths
        self.isSingleBand = isSingleBand
    
    def __getitem__(self, index):
        image = torch.from_numpy(np.vstack(np.load(self.paths[index][0])).astype(float))
        if self.isSingleBand:
            image = image[None,:]
        else:
            image = torch.permute(image, (2,0,1))
        mask_raw = (np.load(self.paths[index][1]))
        maskremap100 = np.where(mask_raw == 100, 0, mask_raw)
        maskremap200 = np.where(maskremap100 == 200, 1, maskremap100)
        mask = torch.from_numpy(np.vstack(maskremap200).astype(float))
        
        #assert image.size == mask.size, \
        #    'Image and mask {name} should be the same size, but are {img.size} and {mask.size}'
        
        return {
            'image': image,
            'mask': mask
        }

    def __len__(self):
        return len(self.paths) 
    

class CustomImageAugmentDataset(Dataset):
    """GTC Code for an augmented dataset class. The class is instantiated with a list of filenames within a directory 
    (created using the list_npy_filenames function). The __getitem__ method pairs up corresponding image-label .npy file
    pairs. If specified, augmentations are also applied to the images with a probability. There is a ~25% chance that 
    no augmentations are applied and a 20% chance of each of the following augmentations: horizontal flip, vertical 
    flip, 90 degree rotation (anti-clockwise & clockwise), 180 degree rotation, random crop. Multiple augmentations are
    applied in sequence. This dataset can then be input to a dataloader."""

    def __init__(self, paths, augmentation):
        self.paths = paths
        self.augmentation = augmentation

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        image = torch.from_numpy(np.vstack(np.load(self.paths[index][0])).astype(float))[None, :]
        mask_raw = (np.load(self.paths[index][1]))
        maskremap100 = np.where(mask_raw == 100, 100, mask_raw) # 0
        maskremap200 = np.where(maskremap100 == 200, 200, maskremap100) # 1
        mask = torch.from_numpy(np.vstack(maskremap200).astype(float))

        if self.augmentation:
            image_pair = self.augment_image(image, mask)
        else:
            image_pair = {'image': image, 'mask': mask}

        return image_pair

    @staticmethod
    def augment_image(original_image, original_mask):

        augmented_image = original_image
        augmented_mask = original_mask

        aug_probabilities = np.random.choice(a=[0, 1], size=4, p=[0.75, 0.25])

        if aug_probabilities[0]: # Horizontal flip
            augment_function = transforms.RandomHorizontalFlip(p=1)
            augmented_image, augmented_mask = augment_function(augmented_image), augment_function(augmented_mask)

        if aug_probabilities[1]: # Vertical flip
            augment_function = transforms.RandomVerticalFlip(p=1)
            augmented_image, augmented_mask = augment_function(augmented_image), augment_function(augmented_mask)

        if  avg_probabilities[2]: # Rotations
            rotations = np.random.choice(a=[1, 2, 3], size=1)
            augmented_image = torch.rot90(augmented_image, k=rotations, dims=[1, 2])
            augmented_mask = torch.rot90(augmented_mask, k=rotations, dims=[0, 1])

        if aug_probabilities[3]: # Random crop (and resize)
            augment_function = transforms.Compose([transforms.RandomCrop(size=256),
                                                   transforms.Resize(512)])
            augmented_image, augmented_mask = augment_function(augmented_image), augment_function(augmented_mask)

        """
        if aug_probabilities[6]: # Gaussian blur
            augment_function = transforms.GaussianBlur(kernel_size=(7,13), sigma=(0.1, 0.2))
            augmented_image, augmented_mask = augment_function(augmented_image), augment_function(augmented_mask)
        """
        return {'image': augmented_image, 'mask': augmented_mask}

In [11]:
""" Dice Loss scoring functions """

import torch
from torch import Tensor


def dice_coeff(input: Tensor, target: Tensor, reduce_batch_first: bool = False, epsilon=1e-6):
    # Average of Dice coefficient for all batches, or for a single mask
    assert input.size() == target.size()
    if input.dim() == 2 and reduce_batch_first:
        raise ValueError(f'Dice: asked to reduce batch but got tensor without batch dimension (shape {input.shape})')

    if input.dim() == 2 or reduce_batch_first:
        inter = torch.dot(input.reshape(-1), target.reshape(-1))
        sets_sum = torch.sum(input) + torch.sum(target)
        if sets_sum.item() == 0:
            sets_sum = 2 * inter

        return (2 * inter + epsilon) / (sets_sum + epsilon)
    else:
        # compute and average metric for each batch element
        dice = 0
        for i in range(input.shape[0]):
            dice += dice_coeff(input[i, ...], target[i, ...])
        return dice / input.shape[0]


def multiclass_dice_coeff(input: Tensor, target: Tensor, reduce_batch_first: bool = False, epsilon=1e-6):
    # Average of Dice coefficient for all classes
    assert input.size() == target.size()
    dice = 0
    for channel in range(input.shape[1]):
        dice += dice_coeff(input[:, channel, ...], target[:, channel, ...], reduce_batch_first, epsilon)

    return dice / input.shape[1]


def dice_loss(input: Tensor, target: Tensor, multiclass: bool = False):
    # Dice loss (objective to minimize) between 0 and 1
    assert input.size() == target.size()
    fn = multiclass_dice_coeff if multiclass else dice_coeff
    return 1 - fn(input, target, reduce_batch_first=True)

In [12]:
""" Model evaluation functions """

import torch
import torch.nn.functional as F
from tqdm import tqdm


def evaluate(net, dataloader, device):
    net.eval()
    num_val_batches = len(dataloader)
    dice_score = 0

    # iterate over the validation set
    for batch in tqdm(dataloader, total=num_val_batches, desc='Validation round', unit='batch', leave=False):
        image, mask_true = batch['image'], batch['mask']
        # move images and labels to correct device and type
        image = image.to(device=device, dtype=torch.float32)
        mask_true = mask_true.to(device=device, dtype=torch.long)
        mask_true = F.one_hot(mask_true, net.n_classes).permute(0, 3, 1, 2).float()

        with torch.no_grad():
            # predict the mask
            mask_pred = net(image)

            # convert to one-hot format
            if net.n_classes == 1:
                mask_pred = (F.sigmoid(mask_pred) > 0.5).float()
                # compute the Dice score
                dice_score += dice_coeff(mask_pred, mask_true, reduce_batch_first=False)
            else:
                mask_pred = F.one_hot(mask_pred.argmax(dim=1), net.n_classes).permute(0, 3, 1, 2).float()
                # compute the Dice score, ignoring background
                dice_score += multiclass_dice_coeff(mask_pred[:, 1:, ...], mask_true[:, 1:, ...], reduce_batch_first=False)

           

    net.train()

    # Fixes a potential division by zero error
    if num_val_batches == 0:
        return dice_score
    return dice_score / num_val_batches

In [13]:
""" Implementation of model """

import argparse
import logging
import sys
from pathlib import Path

import torch
from torch import Tensor
import torch.nn as nn
import torch.nn.functional as F
import wandb
wandb.init(project="my-test-project", entity="mlisaius")

from torch import optim
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

dir_img = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/trainingdata/tiled/')
dir_checkpoint = Path('/mnt/g/Shared drives/2021-gtc-sea-ice/model/checkpoints/')


def train_net(net,
              device,
              epochs: int = 5,
              batch_size: int = 10,
              learning_rate: float = 0.001,
              val_percent: float = 0.1,
              save_checkpoint: bool = True,
              img_scale: float = 0.5,
              amp: bool = False):
    # 1. Create dataset
    img_list = create_npy_list(dir_img)
    dataset = CustomImageDataset(img_list, True)
    
    # 2. Split into train / validation partitions
    n_val = int(len(dataset) * val_percent)
    n_train = len(dataset) - n_val
    train_set, val_set = random_split(dataset, [n_train, n_val], generator=torch.Generator().manual_seed(0))
    
    # 3. Create data loaders
    loader_args = dict(batch_size=batch_size, num_workers=4, pin_memory=True)
    train_loader = DataLoader(train_set, shuffle=True, **loader_args)
    val_loader = DataLoader(val_set, shuffle=False, drop_last=True, **loader_args)
    
    # Initialize logging
    experiment = wandb.init(project='U-Net', resume='allow', anonymous='must')
    experiment.config.update(dict(epochs=epochs, batch_size=batch_size, learning_rate=learning_rate,
                                  val_percent=val_percent, save_checkpoint=save_checkpoint, img_scale=img_scale,
                                  amp=amp))

    logging.info(f'''Starting training:
        Epochs:          {epochs}
        Batch size:      {batch_size}
        Learning rate:   {learning_rate}
        Training size:   {n_train}
        Validation size: {n_val}
        Checkpoints:     {save_checkpoint}
        Device:          {device.type}
        Images scaling:  {img_scale}
        Mixed Precision: {amp}
    ''')

    # 4. Set up the optimizer, the loss, the learning rate scheduler and the loss scaling for AMP
    optimizer = optim.RMSprop(net.parameters(), lr=learning_rate, weight_decay=1e-8, momentum=0.9)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'max', patience=2)  # goal: maximize Dice score
    grad_scaler = torch.cuda.amp.GradScaler(enabled=amp)
    criterion = nn.CrossEntropyLoss()
    global_step = 0
    
    # 5. Begin training
    for epoch in range(epochs):
        net.train()
        epoch_loss = 0
        with tqdm(total=n_train, desc=f'Epoch {epoch + 1}/{epochs}', unit='img') as pbar:
            for batch in train_loader:
                images = batch['image']
                true_masks = batch['mask']
                
                assert images.shape[1] == net.n_channels, \
                    f'Network has been defined with {net.n_channels} input channels, ' \
                    f'but loaded images have {images.shape[1]} channels of {images.shape}. Please check that ' \
                    'the images are loaded correctly.'

                images = images.to(device=device, dtype=torch.float32)
                true_masks = true_masks.to(device=device, dtype=torch.long)

                with torch.cuda.amp.autocast(enabled=amp):
                    masks_pred = net(images)
                    loss = criterion(masks_pred, true_masks) \
                           + dice_loss(F.softmax(masks_pred, dim=1).float(),
                                       F.one_hot(true_masks, net.n_classes).permute(0, 3, 1, 2).float(),
                                       multiclass=True)
                # change number for permute?
                
                optimizer.zero_grad(set_to_none=True)
                grad_scaler.scale(loss).backward()
                grad_scaler.step(optimizer)
                grad_scaler.update()

                pbar.update(images.shape[0])
                global_step += 1
                epoch_loss += loss.item()
                experiment.log({
                    'train loss': loss.item(),
                    'step': global_step,
                    'epoch': epoch
                })
                pbar.set_postfix(**{'loss (batch)': loss.item()})

                # Evaluation round
                division_step = (n_train // (10 * batch_size))
                if division_step > 0:
                    if global_step % division_step == 0:
                        histograms = {}
                        for tag, value in net.named_parameters():
                            tag = tag.replace('/', '.')
                            histograms['Weights/' + tag] = wandb.Histogram(value.data.cpu())
                            histograms['Gradients/' + tag] = wandb.Histogram(value.grad.data.cpu())

                        val_score = evaluate(net, val_loader, device)
                        scheduler.step(val_score)

                        logging.info('Validation Dice score: {}'.format(val_score))
                        experiment.log({
                            'learning rate': optimizer.param_groups[0]['lr'],
                            'validation Dice': val_score,
                            'images': wandb.Image(images[0].cpu()),
                            'masks': {
                                'true': wandb.Image(true_masks[0].float().cpu()),
                                'pred': wandb.Image(torch.softmax(masks_pred, dim=1).argmax(dim=1)[0].float().cpu()),
                            },
                            'step': global_step,
                            'epoch': epoch,
                            **histograms
                        })

        if save_checkpoint:
            Path(dir_checkpoint).mkdir(parents=True, exist_ok=True)
            torch.save(net.state_dict(), str(dir_checkpoint / 'checkpoint_epoch{}.pth'.format(epoch + 1)))
            logging.info(f'Checkpoint {epoch + 1} saved!')


def get_args():
    parser = argparse.ArgumentParser(description='Train the UNet on images and target masks')
    parser.add_argument('--epochs', '-e', metavar='E', type=int, default=5, help='Number of epochs')
    parser.add_argument('--batch-size', '-b', dest='batch_size', metavar='B', type=int, default=1, help='Batch size')
    parser.add_argument('--learning-rate', '-l', metavar='LR', type=float, default=0.00001,
                        help='Learning rate', dest='lr')
    parser.add_argument('--load', '-f', type=str, default=False, help='Load model from a .pth file')
    parser.add_argument('--scale', '-s', type=float, default=0.5, help='Downscaling factor of the images')
    parser.add_argument('--validation', '-v', dest='val', type=float, default=10.0,
                        help='Percent of the data that is used as validation (0-100)')
    parser.add_argument('--amp', action='store_true', default=False, help='Use mixed precision')

    return parser.parse_args()


if __name__ == '__main__':
    args = get_args()

    logging.basicConfig(level=logging.INFO, format='%(levelname)s: %(message)s')
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    logging.info(f'Using device {device}')

    # Change here to adapt to your data
    # n_channels=3 for RGB images
    # n_classes is the number of probabilities you want to get per pixel
    net = UNet(n_channels=1, n_classes=2, bilinear=True)

    logging.info(f'Network:\n'
                 f'\t{net.n_channels} input channels\n'
                 f'\t{net.n_classes} output channels (classes)\n'
                 f'\t{"Bilinear" if net.bilinear else "Transposed conv"} upscaling')

#    if args.load:
#        net.load_state_dict(torch.load(args.load, map_location=device))
#        logging.info(f'Model loaded from {args.load}')

    net.to(device=device)
    try:
        train_net(net=net,
                  epochs=args.epochs,
                  batch_size=args.batch_size,
                  learning_rate=args.lr,
                  device=device,
                  img_scale=args.scale,
                  val_percent=args.val / 100,
                  amp=args.amp)
    except KeyboardInterrupt:
        torch.save(net.state_dict(), 'INTERRUPTED.pth')
        logging.info('Saved interrupt')
        sys.exit(0)

epoch,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
learning rate,███▂▂▂▁▁▁
step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train loss,▁▁▁▁▇▅▁▁▁▁▁▁▁▂▁▁█▁█▁█▁▁███▁██▁▃█▂▁▁▁▁▁▁▁
validation Dice,▁▁▁▁▁▁▁▁▁
epoch,0
learning rate,0.0
step,1609
train loss,0.8909
validation Dice,0.70432


wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


INFO: Using device cpu
INFO: Network:
	1 input channels
	2 output channels (classes)
	Bilinear upscaling


wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


INFO: Starting training:
        Epochs:          5
        Batch size:      1
        Learning rate:   1e-05
        Training size:   1669
        Validation size: 185
        Checkpoints:     True
        Device:          cpu
        Images scaling:  0.5
        Mixed Precision: False
    
Epoch 1/5:   0%|                                                                              | 0/1669 [00:00<?, ?img/s]/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/nn/modules/container.py:141: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)
Validation round: 100%|████████████████████████████████████████

Traceback (most recent call last):
  File "/tmp/ipykernel_252/2428837494.py", line 189, in <module>
    amp=args.amp)
  File "/tmp/ipykernel_252/2428837494.py", line 89, in train_net
    masks_pred = net(images)
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_252/3889472007.py", line 24, in forward
    x3 = self.down2(x2)
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/tmp/ipykernel_252/1271234201.py", line 40, in forward
    return self.maxpool_conv(x)
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/nn/modules/module.py", line 1102, in _call_impl
    return forward_call(*input, **kwargs)
  File "/home/mlisaius/miniconda3/envs/unetenv/lib/python3.7/site-packages/torch/nn/modules/c

TypeError: object of type 'NoneType' has no len()